#### UD02 PRACTICA 3
## REGRESIÓN LINEAL MÚLTIPLE

### Utiliza pandas y carga los datos
fichero "50_startups.csv". Utilizaremos como predictoras las columnas "R&D Spend", "Marqueting
Spend" y "State" y como target usaremos "Profit" (beneficios). Imprime los 5 primeros ejemplos de las
predictoras:

#### Importación de Librerías

In [36]:
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split #para division de los datos

from sklearn.linear_model import LinearRegression #importacion del LinearRegression

import numpy as np #para la obtencion del SSE, SSR..

#### Carga del csv y primeros datos

In [19]:
df = pd.read_csv('50_Startups.csv')

#Seleccionamos las columnas predictoras R&D Spend, Marqueting Spend y State
X = df[['R&D Spend', 'Marketing Spend', 'State']]
y = df['Profit'] #target

#Imprimir los 5 primeros
print(X.head())

   R&D Spend  Marketing Spend       State
0  165349.20        471784.10    New York
1  162597.70        443898.53  California
2  153441.51        407934.54     Florida
3  144372.41        383199.62    New York
4  142107.34        366168.42     Florida


Para realizar regresión lineal no podemos usar una columna categórica como "State".

Codifica sus valores usando el método one-hot-encoder de manera que aparecerán 3 nuevas columnas que corresponden a cada uno de los valores que puede tener la columna original (‘New York’, ‘California’, ‘Florida’) y sus valores estarán a 1 cuando sea ese valor y a 0 cuando no. En vez de hacerlo a mano, vamos a utilizar los objetos sklearn.compose.ColumnTransformer y sklearn.preprocessing.OneHotEncoder

In [20]:
df.info() #Comprobamos que es verdad que es categórica State

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [21]:
#Aplicar One-Hot Encoding a la columna categórica "State"
column_transformer = ColumnTransformer(transformers=[
    ('encoder', OneHotEncoder(), ['State'])
], remainder='passthrough') #todas las demás columnas se mantienen sin cambios

X_encoded = column_transformer.fit_transform(X) #aplica la transformación

#Convertir a DataFrame con nombres de columnas adecuados
encoded_columns = column_transformer.get_feature_names_out()
X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_columns)

#Imprimir los 5 primeros ejemplos de los datos transformados
print("Después de aplicar OneHotEncoder a State")
print(X_encoded_df.head())

Después de aplicar OneHotEncoder a State
   encoder__State_California  encoder__State_Florida  encoder__State_New York  \
0                        0.0                     0.0                      1.0   
1                        1.0                     0.0                      0.0   
2                        0.0                     1.0                      0.0   
3                        0.0                     0.0                      1.0   
4                        0.0                     1.0                      0.0   

   remainder__R&D Spend  remainder__Marketing Spend  
0             165349.20                   471784.10  
1             162597.70                   443898.53  
2             153441.51                   407934.54  
3             144372.41                   383199.62  
4             142107.34                   366168.42  


Como todo es un proceso aleatorio, para poder obtener resultados comparables utilizamos la semilla "123" en todos los procesos.

Divide los datos en train y test dejando el 80% para entrenamiento.

Imprime los 5 primeros ejemplos de **X_train, y_train, X_test** e **y_test:**

In [ ]:
#Dividimos los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_encoded_df, y, test_size=0.2, random_state=123)


#Imprimimos los 5 primeros ejemplos
print("X_train 5 primeras filas:\n", X_train.head())
print("y_train 5 primeras filas:\n", y_train.head())
print("X_test 5 primeras filas:\n", X_test.head())
print("y_test 5 primeras filas:\n", y_test.head())

X_train 5 primeras filas:
     encoder__State_California  encoder__State_Florida  \
21                        0.0                     0.0   
47                        1.0                     0.0   
11                        1.0                     0.0   
41                        0.0                     1.0   
5                         0.0                     0.0   

    encoder__State_New York  remainder__R&D Spend  remainder__Marketing Spend  
21                      1.0              78389.47                   299737.29  
47                      0.0                  0.00                        0.00  
11                      0.0             100671.96                   249744.55  
41                      0.0              27892.92                   164470.71  
5                       1.0             131876.90                   362861.36  
y_train 5 primeras filas:
 21    111313.02
47     42559.73
11    144259.40
41     77798.83
5     156991.12
Name: Profit, dtype: float64
X_test 5 prime

Crea un objeto sklearn.linear_model.LinearRegression y lo entrenas. Calcula su score para datos de train y test

In [34]:
#Creamos y entrenar el modelo de regresión lineal para train y test
model = LinearRegression()
model.fit(X_train, y_train)

#Calculamos el score R2 para train y test
r2_train = model.score(X_train, y_train)
r2_test = model.score(X_test, y_test)
print(f"R2 score (Train): {r2_train:.4f}")
print(f"R2 score (Test): {r2_test:.4f}")

R2 score (Train): 0.9449
R2 score (Test): 0.9658


#### ENTREGA 1

##### a) Captura de imágenes

Ya documentado

##### b) ¿Qué significa en este caso que el score sea aproximadamente del 0.966 en los datos de test?

Significa que el modelo tiene una buena precisión en la predicción del resultado en el conjunto de los datos de prueba


----------------------------------------


Ahora mira los apuntes de la unidad y como se calcula el SSE, SST y SSR, R2
 y R
2 ajustado. Define 3
funciones de Python que calcule el el SSE, SST y SSR. Calcula e imprime sus valores para los datos de
train y test

#### Obtención del SSE, SST, SSR, R2 y R2 ajustado

In [44]:
#Obtención del SSE (Es la suma de Errores al cuadrado)
def calcula_sse(y_real, y_pred):
    return np.sum((y_real - y_pred) **2)

#Obtención del SST (Es la suma total de los cuadrados)
def calcula_sst(y_real):
    return np.sum((y_real - np.mean(y_real)) **2 )

#Obtención del SSR (Es la suma de los cuadrados de la regresión)
def calcula_ssr(y_real, y_pred):
    return calcula_sst(y_real) - calcula_sse(y_real, y_pred)


#Obtener predicciones del modelo
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

#Calculamos los valores para train y test
sse_train = calcula_sse(y_train, y_train_pred)
sst_train = calcula_sst(y_train)
ssr_train = calcula_ssr(y_train, y_train_pred)

sse_test = calcula_sse(y_test, y_test_pred)
sst_test = calcula_sst(y_test)
ssr_test = calcula_ssr(y_test, y_test_pred)

#Calculamos el R2
r2_train = 1 - (sse_train / sst_train)
r2_test = 1 - (sse_test / sst_test)

#Calculamos el R2 ajustado
n_train, p = X_train.shape  # n = número de muestras, p = número de predictores
n_test = X_test.shape[0]

r2_adj_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p - 1))
r2_adj_test = 1 - ((1 - r2_test) * (n_test - 1) / (n_test - p - 1))


#Resultados
print(f"SSE (Train): {sse_train:.4f}, SSE (Test): {sse_test:.4f}")
print(f"SST (Train): {sst_train:.4f}, SST (Test): {sst_test:.4f}")
print(f"SSR (Train): {ssr_train:.4f}, SSR (Test): {ssr_test:.4f}")
print(f"R2 (Train): {r2_train:.4f}, R2 (Test): {r2_test:.4f}")
print(f"R2 Ajustado (Train): {r2_adj_train:.4f}, R2 Ajustado (Test): {r2_adj_test:.4f}")


SSE (Train): 3489621036.3920, SSE (Test): 516139212.6672
SST (Train): 63300608353.7709, SST (Test): 15071605761.1151
SSR (Train): 59810987317.3789, SSR (Test): 14555466548.4479
R2 (Train): 0.9449, R2 (Test): 0.9658
R2 Ajustado (Train): 0.9368, R2 Ajustado (Test): 0.9229
